In [1]:
import scipy
import math
import scipy.sparse as sp
import numpy as np
import networkx as nx
import torch
import sys 
import itertools
from networkx.algorithms.approximation import clique
# sys.path.append("..") 
from utils.data_utils import encode_onehot, normalize, sparse_mx_to_torch_sparse_tensor

build p-order adjacency matrix in equation (1)

In [2]:
flag_add_face = True
dataname = 'cora'

In [4]:
''' extract features, labels and graph from the dataset (ref: https://github.com/tkipf/pygcn) '''

idx_features_labels = np.genfromtxt("./data/cora/cora.content",dtype=np.dtype(str))
features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
labels = encode_onehot(idx_features_labels[:, -1])
# build graph
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}
edges_unordered = np.genfromtxt("./data/cora/cora.cites",dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                    dtype=np.int32).reshape(edges_unordered.shape)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(labels.shape[0], labels.shape[0]),
                    dtype=np.float32)
# directed --> symmetric adjacency matrix
# in binary case, this is equivalent as adj = (adj + adj.T)/2
# if the network is weighted, only a single edge is kept (the one with largest weight).
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [5]:
''' build simplicial complexes from cliques '''
G = nx.from_scipy_sparse_matrix(adj)
# if all nodes in a clique (edge, triangle etc.) have same label, it is considered as a simplex
# note that we remove edges whose ends have different labels
SC = []
SC_labels = []
for clq in nx.clique.find_cliques(G):
    if (labels[clq] == labels[clq][0]).all(): # set of nodes have same label
        SC.append(set(clq))
        SC_labels.append(labels[clq][0])
p_max = max([len(SC[i]) for i in range(len(SC))]) - 1
print('number of edges in graph is', G.number_of_edges())
print('number of simplices is', len(SC))
print('maximum dimension of simlices is', p_max)

number of edges in graph is 5278
number of simplices is 2723
maximum dimension of simlices is 4


In [6]:
# check number of points in the SC
points = set()
for s in SC:
    for ss in s:
        points.add(ss)
print('number of nodes in graph is', G.number_of_nodes())
print('number of points in simplcial complex is', len(points))

number of nodes in graph is 2708
number of points in simplcial complex is 2481


In [7]:
''' build p-chains '''
simplex_chains = [[] for _ in range(p_max+1)]   # simplex_chains[p] is the list of p-chains
simplex_labels = [[] for _ in range(p_max+1)]   # simplex_chains[p] is the list of p-chains' labels
simplex_features = [[] for _ in range(p_max+1)] # simplex_chains[p] is the list of p-chains' features
feat_dense = np.array(features.todense())
for i, s in enumerate(SC):
    simplex_chains[len(s)-1].append(s)
    simplex_labels[len(s)-1].append(SC_labels[i])
    simplex_features[len(s)-1].append(sum(feat_dense[list(s),:],0))
# add 0-simplex to the chains
simplex_chains[0] = [set([ss]) for ss in points]
simplex_labels[0] = labels[list(points),:]
simplex_features[0] = feat_dense[list(points),:]

In [8]:
def add_pseudo(p, chain_p, label_p, feature_p, chain, label, feature):
    """ chain_p: p-chain
        chain: (p-1)-chain
        pseudo: (p-1)-pseudo simplex, subset of p-simplex
        chain_w_pseudo: union (p-1)-chain and (p-1)-pseudo simplex"""
    chain_w_pseudo = chain.copy()
    label_w_pseudo = label.copy()
    feature_w_pseudo = feature.copy()
    for s_idx, s in enumerate(chain_p):
        for i in itertools.combinations(s,p):
            if not set(i) in chain_w_pseudo:
                """pseudo simplex (subset of the p-simplex s) doesn't exist before in the (p-1)-chains
                simply add the pseudo simplex"""
                chain_w_pseudo.append(set(i))
                label_w_pseudo.append(label_p[s_idx])
                feature_w_pseudo.append(feature_p[s_idx])
    return chain_w_pseudo, label_w_pseudo, feature_w_pseudo

In [9]:
''' add faces to lower dimension simplex '''
if flag_add_face:
    # chains with pseudo simplex c^p and s(c^{p+1})
    simplex_chains_all = [[] for _ in range(p_max+1)]
    simplex_labels_all = [[] for _ in range(p_max+1)]
    simplex_features_all = [[] for _ in range(p_max+1)]
    for p in range(p_max+1):
        if p < p_max:
            simplex_chains_all[p], simplex_labels_all[p], simplex_features_all[p] \
            = add_pseudo(p+1, simplex_chains[p+1], simplex_labels[p+1], simplex_features[p+1], 
                simplex_chains[p], simplex_labels[p], simplex_features[p])
        else:
            simplex_chains_all[p], simplex_labels_all[p], simplex_features_all[p] \
            = simplex_chains[p].copy(), simplex_labels[p].copy(), simplex_features[p].copy()

In [10]:
for p in range(p_max+1):
    print(p, len(simplex_chains[p]), len(simplex_chains_all[p]))

0 2481 2481
1 1805 3590
2 758 1294
3 154 183
4 6 6


In [11]:
simplex_features_all_unormalized = simplex_features_all.copy()
for p in range(len(simplex_features_all_unormalized)):
    simplex_features_all_unormalized[p] = np.array(simplex_features_all_unormalized[p])

In [12]:
"""labels to tensor"""
for p in range(len(simplex_labels)):
    simplex_labels[p] = torch.LongTensor(np.where(simplex_labels[p])[1])
    simplex_labels_all[p] = torch.LongTensor(np.where(simplex_labels_all[p])[1])
"""normalize features"""
for p in range(len(simplex_features)):
    simplex_features[p] = torch.FloatTensor(normalize(np.array(simplex_features[p])))
    simplex_features_all[p] = torch.FloatTensor(normalize(np.array(simplex_features_all[p])))

In [13]:
def higher_order_ana(p_max, simplex_chains):
    """build incidence matrices"""
    incidences = [[] for _ in range(p_max+2)]
    incidences[0] = np.zeros((1,len(simplex_chains[0])))           # incidence[0] = [0,...,0] (row)
    incidences[p_max+1] = np.zeros((len(simplex_chains[p_max]),1)) # incidence[p_max+1] = [0,...,0] (column)
    for p in range(1,p_max+1):
        # incidences[p]: chain[p] --> chain[p-1]
        incidences[p] = np.zeros((len(simplex_chains[p-1]),len(simplex_chains[p])))
        for i in range(len(simplex_chains[p-1])):
            for j in range(len(simplex_chains[p])):
                if set(simplex_chains[p-1][i]).issubset(set(simplex_chains[p][j])): incidences[p][i][j] = 1

    """build higher order laplacian matrices"""
    laplacians = [[] for _ in range(p_max+1)] # laplacians[p]: p-order laplacian matrix, p=0,...,p_max
    for p in range(p_max+1):
        laplacians[p] = incidences[p].T @ incidences[p] + incidences[p+1] @ incidences[p+1].T

    """extract higher order adjacency matrices from the laplacians"""
    degrees = [np.diag(np.diag(laplacians[i])) for i in range(len(laplacians))]
    adjacencies = laplacians.copy()
    adj_norm_sp = []
    for p in range(len(adjacencies)):
        a_self = 1 if (p==0 or p== p_max) else 2
        np.fill_diagonal(adjacencies[p],a_self) # add self-loops with weight 2: A = A + 2I_N
        adj_norm = normalize(adjacencies[p]) #D^(-1/2)AD^(1/2)
        adj_norm_sp.append(torch.from_numpy(adj_norm).to_sparse().double()) # convert to sparse tensor
        
    return adjacencies, adj_norm_sp, incidences, laplacians

In [14]:
# adjacencies, adj_norm_sp, incidences, laplacians = higher_order_ana(p_max, simplex_chains)
adjacencies_all, adj_norm_sp_all, incidences_all, laplacians_all = higher_order_ana(p_max, simplex_chains_all)

In [15]:
p = 2
# print(adjacencies[p].shape[0], np.sum(adjacencies[p],axis = (0,1)))
print(adjacencies_all[p].shape[0], np.sum(adjacencies_all[p],axis = (0,1)))
# adjacencies_all[p]

1294 8060.0


In [16]:
for p in range(len(adjacencies_all)):
    adjacencies_all[p] = torch.from_numpy(adjacencies_all[p]).to_sparse().double() ###
    incidences_all[p] = torch.from_numpy(incidences_all[p]).to_sparse().double() ###

In [17]:
torch.save({'adj_p':adjacencies_all, 'adj_norm_sp':adj_norm_sp_all, 
        'simplex_features':simplex_features_all_unormalized, 'simplex_labels':simplex_labels_all, 
        'simplex_chains': simplex_chains_all, 'simplex_incidences': incidences_all},
       './data/{}/pseudo/input_s_0.pt'.format(dataname))
torch.save({'adj_norm_sp':adj_norm_sp_all,'simplex_features':simplex_features_all,
        'simplex_labels':simplex_labels_all, 'simplex_chains': simplex_chains_all},
       './data/{}/pseudo/input_s.pt'.format(dataname))

build full adjacency matrix in equation (3)

In [18]:
dataname = 'cora'
path = './data/{}/pseudo/'.format(dataname)
sx = torch.load(path + 'input_s_0.pt')

adj_p = sx['adj_p']
feat_p = sx['simplex_features']
label_p = sx['simplex_labels']
chain_p = sx['simplex_chains']
incidence_p = sx['simplex_incidences']

In [19]:
# connection weight (strength) for simplex self connection; 
# simplex self connection within same dimension; 
# between different dimension (i.e., whether is face and coface)
w_self_con = 1
w_simplex_con = 1
w_face_con = 0.5

for p in range(len(adj_p)):
    adj_p[p] = adj_p[p].to_dense()
    incidence_p[p] = incidence_p[p].to_dense()
    adj_p[p] *= adj_p[p] * w_simplex_con
    adj_p[p].fill_diagonal_(w_self_con)

In [20]:
# build connection of all simplex with different dimension
N_p = [feat_p[p].shape[0] for p in range(len(feat_p))]
N = sum(N_p)
adj = torch.zeros((N,N))
for i in range(len(adj_p)):
    adj[sum(N_p[:i]):sum(N_p[:i+1]),sum(N_p[:i]):sum(N_p[:i+1])] = adj_p[i]
    if i < len(adj_p) - 1:
        adj[sum(N_p[:i]):sum(N_p[:i+1]),sum(N_p[:i+1]):sum(N_p[:i+2])] \
        = w_face_con * incidence_p[i+1]
        adj[sum(N_p[:i+1]):sum(N_p[:i+2]),sum(N_p[:i]):sum(N_p[:i+1])] \
        = w_face_con * incidence_p[i+1].T      
adj_sp = adj.to_sparse().double()

In [21]:
for p in range(len(feat_p)):
    feat_p[p] = torch.FloatTensor(feat_p[p])
    
feat = torch.cat((feat_p),0)
feat = torch.FloatTensor(normalize(feat))
label = torch.cat((label_p),0)

In [22]:
torch.save({'adj':adj_sp, 'feat': feat, 'label': label}, path+'input_sc.pt')